In [74]:
!pip install keras

import math
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten

from sklearn.model_selection import train_test_split


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load Cleaned Data

In [40]:
data = pd.read_csv("cleaned_data.csv", sep=",")

columns = data.columns

print(data.columns)

Index(['Unnamed: 0', 'p1/head:tx', 'p1/head:ty', 'p1/head:tz', 'p1/hand_l:tx',
       'p1/hand_l:ty', 'p1/hand_l:tz', 'p1/hand_r:tx', 'p1/hand_r:ty',
       'p1/hand_r:tz', 'p1/foot_l:tx', 'p1/foot_l:ty', 'p1/foot_l:tz',
       'p1/foot_r:tx', 'p1/foot_r:ty', 'p1/foot_r:tz', 'p1/hip_l:tx',
       'p1/hip_l:ty', 'p1/hip_l:tz', 'p1/hip_r:tx', 'p1/hip_r:ty',
       'p1/hip_r:tz', 'p1/shoulder_l:tx', 'p1/shoulder_l:ty',
       'p1/shoulder_l:tz', 'p1/shoulder_r:tx', 'p1/shoulder_r:ty',
       'p1/shoulder_r:tz', 'p1/shoulder_c:tx', 'p1/shoulder_c:ty',
       'p1/shoulder_c:tz', 'p1/knee_l:tx', 'p1/knee_l:ty', 'p1/knee_l:tz',
       'p1/knee_r:tx', 'p1/knee_r:ty', 'p1/knee_r:tz', 'p1/elbow_l:tx',
       'p1/elbow_l:ty', 'p1/elbow_l:tz', 'p1/elbow_r:tx', 'p1/elbow_r:ty',
       'p1/elbow_r:tz', 'p1/head:dx', 'p1/head:dy', 'p1/head:dz',
       'p1/hand_l:dx', 'p1/hand_l:dy', 'p1/hand_l:dz', 'p1/hand_r:dx',
       'p1/hand_r:dy', 'p1/hand_r:dz', 'p1/foot_l:dx', 'p1/foot_l:dy',
       'p1/foot_

In [41]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

dataset = data.values
dataset = dataset.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [45]:
datasets_split = np.split(np.array(dataset), 23)

print(len(datasets_split[0]))

200


In [86]:
# Create proper datasets for each individual's sample
xs, ys = [], []

def add_to_dataset(frames):
    classification = frames[0][-1]
    if classification == 1.0:
        ys.append([1.0, 0.0])
    else:
        ys.append([0.0, 1.0])
    
    velocities = []
    for row in frames:
        velocities.append(row[-44:-2])
        
    xs.append(velocities)


# create the dataset
for frames in datasets_split:
    add_to_dataset(frames)


print(ys)
print(len(xs[0]))
print(len(xs[0][0]))
print(xs[0][0])

[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]
200
42
[0.62871313 0.36946708 0.5736951  0.59707475 0.35386896 0.4380762
 0.37094203 0.34215742 0.40209943 0.6373383  0.44952083 0.40437236
 0.46741185 0.45592755 0.47824284 0.46967116 0.50029474 0.5601061
 0.49721682 0.36157554 0.5602745  0.39169955 0.4026038  0.50698006
 0.5636982  0.29688847 0.54138064 0.6532098  0.38921413 0.57773304
 0.5262114  0.50660866 0.557436   0.41649243 0.43784288 0.54650044
 0.5827864  0.59531236 0.36655694 0.6909118  0.31066352 0.5810995 ]


In [55]:
# TODO: split into train and test...

## Train

In [93]:
# reshape input to be [samples, timesteps, features]
x_train = np.array(xs)
y_train = np.array(ys)

fakex_train = np.random.random((100, timesteps, data_dim))
fakey_train = np.random.random((100, num_classes))

print(type(y_train))

print(y_train.shape)
print(fakey_train.shape)
print(y_train)
print(x_train.shape)
print(fakex_train.shape)


<class 'numpy.ndarray'>
(23, 2)
(100, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
(23, 200, 42)
(100, 200, 42)


In [96]:
data_dim = 42
timesteps = 200
num_classes = 2

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 42
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=3, epochs=10)


# model.predict(x_test)

Epoch 1/10
23/23 [==============================] - 10s 418ms/step - loss: 0.7184 - acc: 0.4783
Epoch 2/10
23/23 [==============================] - 5s 223ms/step - loss: 0.6913 - acc: 0.5217
Epoch 3/10
23/23 [==============================] - 5s 223ms/step - loss: 0.6902 - acc: 0.4783
Epoch 4/10
23/23 [==============================] - 5s 232ms/step - loss: 0.6957 - acc: 0.4783
Epoch 5/10
23/23 [==============================] - 5s 222ms/step - loss: 0.6913 - acc: 0.5217
Epoch 6/10
23/23 [==============================] - 6s 244ms/step - loss: 0.6893 - acc: 0.5217
Epoch 7/10
23/23 [==============================] - 5s 225ms/step - loss: 0.6936 - acc: 0.5217
Epoch 8/10
23/23 [==============================] - 5s 221ms/step - loss: 0.6875 - acc: 0.5217
Epoch 9/10
23/23 [==============================] - 5s 228ms/step - loss: 0.6863 - acc: 0.6087
Epoch 10/10
23/23 [==============================] - 5s 225ms/step - loss: 0.6858 - acc: 0.5217


In [97]:
model.predict(x_train)

array([[0.48443928, 0.5155606 ],
       [0.48224613, 0.51775384],
       [0.48893583, 0.5110642 ],
       [0.4826293 , 0.5173707 ],
       [0.44891062, 0.55108935],
       [0.49425253, 0.50574744],
       [0.47287473, 0.52712524],
       [0.49062234, 0.50937766],
       [0.4691325 , 0.53086746],
       [0.47541767, 0.5245824 ],
       [0.49243382, 0.50756615],
       [0.42587626, 0.57412374],
       [0.44929534, 0.55070466],
       [0.50357366, 0.49642634],
       [0.48033652, 0.51966345],
       [0.4515187 , 0.5484812 ],
       [0.4908459 , 0.50915414],
       [0.43472576, 0.56527424],
       [0.4599356 , 0.54006433],
       [0.4632047 , 0.5367953 ],
       [0.4712061 , 0.52879393],
       [0.47320852, 0.5267915 ],
       [0.47662145, 0.52337855]], dtype=float32)

## Another attempt at sequence classification with LSTM: (binary-crossentropy?)


In [102]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

max_features = 1024

model = Sequential()
model.add(LSTM(128, input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 42
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
# score = model.evaluate(x_test, y_test, batch_size=16)

Epoch 1/10
23/23 [==============================] - 6s 263ms/step - loss: 0.7610 - acc: 0.4783
Epoch 2/10
23/23 [==============================] - 2s 91ms/step - loss: 0.7190 - acc: 0.5000
Epoch 3/10
23/23 [==============================] - 2s 89ms/step - loss: 0.6745 - acc: 0.6304
Epoch 4/10
23/23 [==============================] - 2s 92ms/step - loss: 0.6796 - acc: 0.5435
Epoch 5/10
23/23 [==============================] - 2s 86ms/step - loss: 0.7164 - acc: 0.4783
Epoch 6/10
23/23 [==============================] - 2s 75ms/step - loss: 0.7201 - acc: 0.5217
Epoch 7/10
23/23 [==============================] - 2s 75ms/step - loss: 0.7331 - acc: 0.4783
Epoch 8/10
23/23 [==============================] - 2s 76ms/step - loss: 0.6500 - acc: 0.6957
Epoch 9/10
23/23 [==============================] - 2s 88ms/step - loss: 0.6858 - acc: 0.6087
Epoch 10/10
23/23 [==============================] - 2s 74ms/step - loss: 0.7457 - acc: 0.3696


In [105]:
model.predict(x_train)

array([[0.57443154, 0.46600696],
       [0.5688721 , 0.46644348],
       [0.57233405, 0.4652578 ],
       [0.5743616 , 0.46577564],
       [0.56688404, 0.46985695],
       [0.5766037 , 0.4629998 ],
       [0.57338226, 0.46404663],
       [0.574982  , 0.46421918],
       [0.5715662 , 0.46870497],
       [0.5684801 , 0.46452418],
       [0.577626  , 0.46401316],
       [0.5569087 , 0.473774  ],
       [0.5665229 , 0.4684041 ],
       [0.58088815, 0.46124077],
       [0.5698734 , 0.46488866],
       [0.56363624, 0.46993223],
       [0.57493204, 0.46398753],
       [0.5616182 , 0.47042862],
       [0.568425  , 0.4673265 ],
       [0.5678428 , 0.46983978],
       [0.5703074 , 0.46787316],
       [0.5725456 , 0.4676943 ],
       [0.57270354, 0.46331975]], dtype=float32)

In [104]:
score = model.evaluate(x_train, y_train, batch_size=16)

print(score)

23/23 [==============================] - 1s 30ms/step
[0.701069712638855, 0.4782608695652174]


In [109]:
from keras import utils
xt = np.random.random((100, 100, 100, 3))
yt = utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

In [111]:
xt.shape

(100, 100, 100, 3)

In [118]:
yt

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 